# ROADMAP (TODO -> Apagar depois)

1. Definir a diferenca metodologica entre as duas fontes
2. Tornar os dados comparaveis
3. Quantificar a diferenca entre os dois sistemas (Tom e Gov) em horas e percentual
4. Padroes ou distorcoes por escola, funcao ou mes

Passos
1. Cruzar dados -> Tom e Base Gov (entender as métricas de carga horária)
    1.1 Diferença entre regencia e nao regencia
    1.2 Demissão (como avaliar)
    1.3 Entender quantas horas, cada professor, por mes, trabalhou (cruzando os dados entre estimativa Tom e 
    estimativa GOV)
    1.4 Identificar discrepencias (horas faltantes)
    
2. Calcular pagamento por mês a cada professor baseado no cáculo de regencia e não regencia e por escola
3. Identificar quantidade de horas totais de professores em cada escola por tipo de funcao, valor, e CH


# **O desafio**

Atualmente, a Tom atua na gestão da educação pública em parceria com o gorverno. O desafio parte da necessidade da Tom identificar e analisar as principais diferenças entre um de seus serviços, a contratação e gestão de carga horária de professores. 

In [475]:
# Manipulação dos dados
import pandas as pd
import numpy as np

# Visualização dos dados
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from warnings import filterwarnings

filterwarnings('ignore')

# Manipulacao de arquivos e caminhos
from pathlib import Path

ROOT_DIR = Path().cwd().parent
DATA_DIR = ROOT_DIR / "data"
FILE_PATH = DATA_DIR / "data.xlsx"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set_theme(
    context="notebook",
    style="whitegrid",
    palette="colorblind",
    font="Arial",
    font_scale=1.0
)
plt.rc("figure", figsize=(12, 6), dpi=120)


In [476]:
sheets = pd.ExcelFile(FILE_PATH)

names = sheets.sheet_names

names.remove('Auxiliares>>>>')

In [477]:
SHEET_KEYS = {
    i: name
    for i, name in enumerate(names)
}

SHEET_KEYS

{0: 'Resumo Executivo',
 1: 'Análise CH por Escola',
 2: 'BD_Tom Folha',
 3: 'BD_SistemaGov',
 4: 'Valores por escola',
 5: 'Página30'}

In [478]:
hc_per_school = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[1])

print(hc_per_school.shape)
hc_per_school.head()

(128, 9)


,Mês,Escola,CÓDIGO ESCOLA,Valor Unitário Carga Horária,CH Regência,CH Sem Regência,Valor Regência,Valor Sem Regência,Valor CH Mensal
0,janeiro,ESCOLA A,ABC1,NaN,NaN,NaN,NaN,NaN,NaN
1,janeiro,ESCOLA B,ABC2,NaN,NaN,NaN,NaN,NaN,NaN
2,janeiro,ESCOLA C,ABC3,NaN,NaN,NaN,NaN,NaN,NaN
3,janeiro,ESCOLA D,ABC4,NaN,NaN,NaN,NaN,NaN,NaN
4,janeiro,ESCOLA E,ABC5,NaN,NaN,NaN,NaN,NaN,NaN


In [479]:
tom_payroll = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[2])

print(tom_payroll.shape)
tom_payroll.head()

(3085, 12)


,Mês,Escola,CodEscola,Nome,CodFuncionario,SITUACAO,ADM,DESCRICAO_CARGO,DESCRICAO_VERBA,Horas Mes,Categoria,Dt demissão
0,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,COORDENADOR(A) DE CURSO,HORAS,70.0,NAO REGENCIA,NaT
1,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,45.0,REGENCIA,NaT
2,Fevereiro,ESCOLA A,ABC1,Márcio Wagner,101.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,27.0,REGENCIA,NaT
3,Fevereiro,ESCOLA A,ABC1,Igor Souza Quirino,102.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,49.5,REGENCIA,NaT
4,Fevereiro,ESCOLA A,ABC1,João Miguel Oliveira Vieira,103.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,40.5,REGENCIA,NaT


In [480]:
gov_payroll = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[3])

print(gov_payroll.shape)
gov_payroll.head()

(6780, 14)


,Mês,Escola,CodEscola,Nome Funcionario,CodFuncionario,Tipo,DisciplinaFuncao,DisciplinaFuncaoDescricao,PeriodoIni,PeriodoFim,DiasTrabalhados,CHSuprimento,CHDiaria,CHPeriodo
0,Fevereiro,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6279,LITER ART E MOVIMENTO I,2025-02-03,2025-02-28,26,8,1.142857,29.714282
1,Fevereiro,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6363,ELETIVA I,2025-02-03,2025-02-28,26,4,0.571428,14.857128
2,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,704,ARTE,2025-02-03,2025-02-28,26,8,1.142857,29.714282
3,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,732,ARTE II,2025-02-03,2025-02-28,26,2,0.285714,7.428564
4,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,5641,PROJETO DE VIDA I,2025-02-03,2025-02-28,26,25,3.571428,92.857128


In [481]:
school_values = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[4])

print(school_values.shape)
school_values.head()

(32, 3)


,Escola,CodEscola,Valor Unitário Carga Horária
0,ESCOLA Z,ABC23,23.09
1,ESCOLA N,ABC14,29.27
2,ESCOLA O,ABC15,29.27
3,ESCOLA E,ABC5,29.27
4,ESCOLA F,ABC6,27.21


In [482]:
teachers_match = pd.read_excel(FILE_PATH, sheet_name=SHEET_KEYS[5])

teachers_match.head()

,UNIQUE,Unnamed: 1,Unnamed: 2,conit,Unnamed: 4,UNIQUE.1,CÓDIGO ESCOLA,Escola
0,61008303100,100,Andressa Aguiar,1,NaN,41128656.0,ABC1,ESCOLA A
1,9966162925,101,Márcio Wagner,1,NaN,41129792.0,ABC2,ESCOLA B
2,638731925,102,Igor Souza Quirino,1,NaN,41131479.0,ABC3,ESCOLA C
3,5743855927,103,João Miguel Oliveira Vieira,1,NaN,41131908.0,ABC4,ESCOLA D
4,2334064920,104,Dennis Oshima,1,NaN,41135784.0,ABC5,ESCOLA E


Limpeza das tabelas

In [483]:
print(hc_per_school[['Mês', 'Escola']].info())

grouped_schools = hc_per_school.groupby('Mês')['Escola'].count()

grouped_schools

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Mês     128 non-null    object
 1   Escola  128 non-null    object
dtypes: object(2)
memory usage: 2.1+ KB
None


Mês
abril        32
fevereiro    32
janeiro      32
março        32
Name: Escola, dtype: int64

In [484]:
hc_per_school = hc_per_school.rename(columns={
    'Mês': 'mes',
    'Escola': 'escola',
    'CÓDIGO ESCOLA': 'codigo_escola',
    'Valor Unitário Carga Horária': 'valor_unitario_carga_horaria',
    'CH Regência': 'ch_regencia',
    'Valor Regência': 'valor_regencia',
    'Valor Sem Regência': 'valor_sem_regencia',
    'Valor CH Mensal': 'valor_ch_mensal',
})

hc_per_school.columns

Index(['mes', 'escola', 'codigo_escola', 'valor_unitario_carga_horaria',
       'ch_regencia', 'CH Sem Regência', 'valor_regencia',
       'valor_sem_regencia', 'valor_ch_mensal'],
      dtype='object')

In [485]:
tom_payroll = tom_payroll.rename(columns={
    'Mês': 'mes',
    'Escola': 'escola',
    'CodEscola': 'codigo_escola',
    'Nome': 'nome',
    'CodFuncionario': 'codigo_funcionario',
    'SITUACAO': 'situacao',
    'ADM': 'data_admissao',
    'DESCRICAO_CARGO': 'descricao_cargo',
    'DESCRICAO_VERBA': 'descricao_verba',
    'Horas Mes': 'ch_total',
    'Categoria': 'categoria',
    'Dt demissão': 'data_demissao',
})

print(tom_payroll.info())
print(tom_payroll.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3085 entries, 0 to 3084
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   mes                 3085 non-null   object        
 1   escola              3085 non-null   object        
 2   codigo_escola       3085 non-null   object        
 3   nome                3078 non-null   object        
 4   codigo_funcionario  3078 non-null   float64       
 5   situacao            3085 non-null   object        
 6   data_admissao       1964 non-null   datetime64[ns]
 7   descricao_cargo     3085 non-null   object        
 8   descricao_verba     3085 non-null   object        
 9   ch_total            3085 non-null   float64       
 10  categoria           3085 non-null   object        
 11  data_demissao       189 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 289.3+ KB
None
       codigo_funcionario      

In [486]:
tom_payroll.isna().sum()

mes                      0
escola                   0
codigo_escola            0
nome                     7
codigo_funcionario       7
situacao                 0
data_admissao         1121
descricao_cargo          0
descricao_verba          0
ch_total                 0
categoria                0
data_demissao         2896
dtype: int64

In [487]:
tom_payroll[(tom_payroll['nome'].isna()) | (tom_payroll['codigo_funcionario'].isna())]

,mes,escola,codigo_escola,nome,codigo_funcionario,situacao,data_admissao,descricao_cargo,descricao_verba,ch_total,categoria,data_demissao
1994,Abril,ESCOLA A,ABC1,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT
2047,Abril,ESCOLA G,ABC7,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.9,REGENCIA,NaT
2640,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,18.0,REGENCIA,NaT
2641,Abril,ESCOLA W,ABC21,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,20.0,REGENCIA,NaT
2800,Abril,ESCOLA Z,ABC23,NaN,NaN,Trabalhando,NaT,COORDENADOR(A) PEDAGOGICO,HORAS,100.0,NAO REGENCIA,NaT
2859,Abril,ESCOLA J,ABC10,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,14.7,REGENCIA,NaT
3048,Abril,ESCOLA M,ABC13,NaN,NaN,Trabalhando,NaT,PROFESSOR(A),HORAS,12.6,REGENCIA,NaT


In [488]:
tom_payroll[['nome', 'codigo_funcionario']] = tom_payroll[['nome', 'codigo_funcionario']].fillna('SEM INFO')


faltantes = tom_payroll[(tom_payroll['nome'] == 'SEM INFO') | (tom_payroll['codigo_funcionario'] == 'SEM INFO')]

faltantes


,mes,escola,codigo_escola,nome,codigo_funcionario,situacao,data_admissao,descricao_cargo,descricao_verba,ch_total,categoria,data_demissao
1994,Abril,ESCOLA A,ABC1,SEM INFO,SEM INFO,Trabalhando,NaT,PROFESSOR(A),HORAS,54.0,REGENCIA,NaT
2047,Abril,ESCOLA G,ABC7,SEM INFO,SEM INFO,Trabalhando,NaT,PROFESSOR(A),HORAS,18.9,REGENCIA,NaT
2640,Abril,ESCOLA W,ABC21,SEM INFO,SEM INFO,Trabalhando,NaT,PROFESSOR(A),HORAS,18.0,REGENCIA,NaT
2641,Abril,ESCOLA W,ABC21,SEM INFO,SEM INFO,Trabalhando,NaT,PROFESSOR(A),HORAS,20.0,REGENCIA,NaT
2800,Abril,ESCOLA Z,ABC23,SEM INFO,SEM INFO,Trabalhando,NaT,COORDENADOR(A) PEDAGOGICO,HORAS,100.0,NAO REGENCIA,NaT
2859,Abril,ESCOLA J,ABC10,SEM INFO,SEM INFO,Trabalhando,NaT,PROFESSOR(A),HORAS,14.7,REGENCIA,NaT
3048,Abril,ESCOLA M,ABC13,SEM INFO,SEM INFO,Trabalhando,NaT,PROFESSOR(A),HORAS,12.6,REGENCIA,NaT


In [489]:
gov_payroll = gov_payroll.rename(columns={
    'Mês': 'mes',
    'Escola': 'escola',
    'CodEscola': 'codigo_escola',
    'Nome Funcionario': 'nome_funcionario',
    'CodFuncionario': 'codigo_funcionario',
    'Tipo': 'categoria',
    'DisciplinaFuncao': 'disciplina_funcao',
    'DisciplinaFuncaoDescricao': 'disciplina_funcao_descricao',
    'PeriodoIni': 'periodo_ini',
    'PeriodoFim': 'periodo_fim',
    'DiasTrabalhados': 'dias_trabalhados',
    'CHSuprimento': 'ch_suprimento',
    'CHDiaria': 'ch_diaria',
    'CHPeriodo': 'ch_periodo',
})


print(gov_payroll.info())
print(gov_payroll.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   mes                          6780 non-null   object        
 1   escola                       6780 non-null   object        
 2   codigo_escola                6780 non-null   object        
 3   nome_funcionario             6780 non-null   object        
 4   codigo_funcionario           6780 non-null   int64         
 5   categoria                    6780 non-null   object        
 6   disciplina_funcao            6780 non-null   int64         
 7   disciplina_funcao_descricao  6780 non-null   object        
 8   periodo_ini                  6780 non-null   datetime64[ns]
 9   periodo_fim                  6780 non-null   datetime64[ns]
 10  dias_trabalhados             6780 non-null   int64         
 11  ch_suprimento                6780 non-null 

In [490]:
gov_payroll.isna().sum()

mes                            0
escola                         0
codigo_escola                  0
nome_funcionario               0
codigo_funcionario             0
categoria                      0
disciplina_funcao              0
disciplina_funcao_descricao    0
periodo_ini                    0
periodo_fim                    0
dias_trabalhados               0
ch_suprimento                  0
ch_diaria                      0
ch_periodo                     0
dtype: int64

In [491]:
school_values = school_values.rename(columns={
    'Escola': 'escola',
    'CodEscola': 'codigo_escola',
    'Valor Unitário Carga Horária': 'valor_unitario_carga_horaria',
})

print(school_values.info())
print(school_values.describe())
print(school_values.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   escola                        32 non-null     object 
 1   codigo_escola                 32 non-null     object 
 2   valor_unitario_carga_horaria  32 non-null     float64
dtypes: float64(1), object(2)
memory usage: 900.0+ bytes
None
       valor_unitario_carga_horaria
count                     32.000000
mean                      26.765000
std                        2.956628
min                       23.090000
25%                       23.090000
50%                       27.210000
75%                       29.270000
max                       31.390000
escola                          0
codigo_escola                   0
valor_unitario_carga_horaria    0
dtype: int64


In [492]:
teachers_match = teachers_match.rename(columns={
    'Unnamed: 1': 'cod_funcionario',
    'Unnamed: 2': 'nome_funcionario',
    'conit': 'conit',
    'Unnamed: 4': 'empty',
    'CÓDIGO ESCOLA': 'codigo_escola',
    'Escola': 'escola',
})

teachers_match = teachers_match.drop(columns=['empty', 'conit', 'UNIQUE.1', 'codigo_escola', 'escola'])

print(teachers_match.columns)
print(teachers_match.info())
print(teachers_match.isna().sum())

Index(['UNIQUE', 'cod_funcionario', 'nome_funcionario'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   UNIQUE            1686 non-null   int64 
 1   cod_funcionario   1686 non-null   int64 
 2   nome_funcionario  1686 non-null   object
dtypes: int64(2), object(1)
memory usage: 39.6+ KB
None
UNIQUE              0
cod_funcionario     0
nome_funcionario    0
dtype: int64


Cálculos básicos


Sabemos que a carga horária total de Tom é calculada com base na carga horária semanal x fator do cargo. Além disso, a carga horária semanal é estimada com base carga horária diária calculada pelo Governo vezes 7 (número de dias da semana). Diante disso, podemos decompor a carga horária de Tom em semanas e calcular a carga horária do Governo em semanas, para podermos ter uma métrica comparável entre as duas tabelas.

In [493]:
tom_payroll['fator'] = tom_payroll['categoria'].map({
    'REGENCIA': 4.5,
    'NAO REGENCIA': 5
})

tom_payroll['ch_semanal'] = tom_payroll['ch_total'] / tom_payroll['fator']

tom_payroll.head()

,mes,escola,codigo_escola,nome,codigo_funcionario,situacao,data_admissao,descricao_cargo,descricao_verba,ch_total,categoria,data_demissao,fator,ch_semanal
0,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,COORDENADOR(A) DE CURSO,HORAS,70.0,NAO REGENCIA,NaT,5.0,14.0
1,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,45.0,REGENCIA,NaT,4.5,10.0
2,Fevereiro,ESCOLA A,ABC1,Márcio Wagner,101.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,27.0,REGENCIA,NaT,4.5,6.0
3,Fevereiro,ESCOLA A,ABC1,Igor Souza Quirino,102.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,49.5,REGENCIA,NaT,4.5,11.0
4,Fevereiro,ESCOLA A,ABC1,João Miguel Oliveira Vieira,103.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,40.5,REGENCIA,NaT,4.5,9.0


In [494]:
gov_payroll['ch_semanal'] = np.round(gov_payroll['ch_diaria'] * 7, 2)

gov_payroll.head()

,mes,escola,codigo_escola,nome_funcionario,codigo_funcionario,categoria,disciplina_funcao,disciplina_funcao_descricao,periodo_ini,periodo_fim,dias_trabalhados,ch_suprimento,ch_diaria,ch_periodo,ch_semanal
0,Fevereiro,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6279,LITER ART E MOVIMENTO I,2025-02-03,2025-02-28,26,8,1.142857,29.714282,8.0
1,Fevereiro,ESCOLA Z,ABC23,Bruna Diniz,680,REGENCIA,6363,ELETIVA I,2025-02-03,2025-02-28,26,4,0.571428,14.857128,4.0
2,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,704,ARTE,2025-02-03,2025-02-28,26,8,1.142857,29.714282,8.0
3,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,732,ARTE II,2025-02-03,2025-02-28,26,2,0.285714,7.428564,2.0
4,Fevereiro,ESCOLA Z,ABC23,Yolanda Sales,681,REGENCIA,5641,PROJETO DE VIDA I,2025-02-03,2025-02-28,26,25,3.571428,92.857128,25.0


In [495]:
more_than_1_school = tom_payroll.groupby(['mes', 'codigo_funcionario', 'categoria'])['codigo_escola'].count().reset_index()

(more_than_1_school['codigo_escola'] > 1).sum()

np.int64(372)

Existem professores que trabalham em mais de uma escola e analisando os dados percebemos que professores que possuem multiplos cargos, isto é, lecionam em multiplas materias. Este é um impasse a ser resolvido, visto que podemos ter professores que lecionam em quantidades de dias diferentes para cada matéria. Por isso, vamos calcular o CH Real do Governo utilizando o CH diario * Dias trabalhados e comparar se todos os resultados batem

In [496]:
gov_payroll_copy = gov_payroll.copy()

ch_real_gov = np.round(gov_payroll_copy['ch_diaria'] * gov_payroll_copy['dias_trabalhados'], 3)

(ch_real_gov - np.round(gov_payroll_copy['ch_periodo'], 3)).sum()

np.float64(0.0)

Nao existe discrepancia na base do governo entre os dados de Ch total

In [ ]:
tom_payroll['ch_diaria'] = tom_payroll['ch_semanal'] / 7

tom_payroll.head()

,mes,escola,codigo_escola,nome,codigo_funcionario,situacao,data_admissao,descricao_cargo,descricao_verba,ch_total,categoria,data_demissao,fator,ch_semanal,ch_diaria
0,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,COORDENADOR(A) DE CURSO,HORAS,70.0,NAO REGENCIA,NaT,5.0,14.0,2.000000
1,Fevereiro,ESCOLA A,ABC1,Andressa Aguiar,100.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,45.0,REGENCIA,NaT,4.5,10.0,1.428571
2,Fevereiro,ESCOLA A,ABC1,Márcio Wagner,101.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,27.0,REGENCIA,NaT,4.5,6.0,0.857143
3,Fevereiro,ESCOLA A,ABC1,Igor Souza Quirino,102.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,49.5,REGENCIA,NaT,4.5,11.0,1.571429
4,Fevereiro,ESCOLA A,ABC1,João Miguel Oliveira Vieira,103.0,Trabalhando,2025-02-03,PROFESSOR(A),HORAS,40.5,REGENCIA,NaT,4.5,9.0,1.285714


In [502]:
duplicatas = (
    gov_payroll
    .groupby(['mes','codigo_escola','codigo_funcionario','disciplina_funcao'])
    .size()
    .reset_index(name='n_registros')
    .query('n_registros > 1')
)

print(len(duplicatas))
duplicatas

301


,mes,codigo_escola,codigo_funcionario,disciplina_funcao,n_registros
93,Abril,ABC10,729,401,4
94,Abril,ABC10,729,2385,2
203,Abril,ABC12,817,704,4
232,Abril,ABC12,1727,401,2
234,Abril,ABC12,1730,1347,3
235,Abril,ABC12,1730,6210,2
236,Abril,ABC12,1730,6460,2
243,Abril,ABC13,755,6460,2
245,Abril,ABC13,850,501,2
246,Abril,ABC13,850,2201,2
